In [19]:
import pandas as pd
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

## 1. Import data

In [20]:
data = pd.read_excel (r'./T4.xlsx')

#### Display the the number of empty values in the dataset

In [32]:
def total_of_missing_values(data):
    missing_data = data.isna().sum().sum()
    print("\nNumber of NaN values:", missing_data)
    #return missing_data

#### Display the the number and the percent of missing values for each column in the dataset

In [26]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (100*data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data[missing_data['Percent']>0])

#### Show the dataset shape

In [35]:
def show_data_dimensions(data):
    print('number of rows : '+str(data.shape[0])+', number of columns : '+str(data.shape[1]))
    #return data.shape[0]

#### Clean the data by dropping the duplicates rows and also by filling the empty values with previous values

In [37]:
def preprocessing(original_data):

    clean_data = original_data.copy()

    ### drop duplicate rows
    clean_data.drop_duplicates(keep='first', inplace=True)

    ### fill missing data and drop the remaing
    clean_data = clean_data.fillna(method='ffill')
    clean_data.dropna(axis=0, inplace=True)
    return clean_data

In [48]:
total_of_missing_values(data)


Number of NaN values: 1010258


In [49]:
missing_data(data)

                                     Total    Percent
function_call_interrupts_cpu0        68773  44.080453
function_call_interrupts_sum_cpu123  68773  44.080453
companion_sum_cpu123                 63667  40.807732
SLIMBUS_sum_cpu123                   63667  40.807732
volume_up_sum_cpu123                 63666  40.807092
...                                    ...        ...
Battery_online                          44   0.028202
Battery_level                           44   0.028202
Battery_invalid_charger                 44   0.028202
Battery_icon_small                      44   0.028202
Battery_health                          44   0.028202

[113 rows x 2 columns]


In [50]:
show_data_dimensions(data)

number of rows : 156017, number of columns : 116


## Clean the data

In [51]:
data2 = preprocessing(data)

In [52]:
#Convert all the data to string so that we can use combination of columns
data2 = data2.applymap(str)

In [53]:
#concate columns to use them for recommendation
data2['version_Battery_level'] = data2['Version'] +"_"+data2['Battery_level']

In [54]:
###This bloc is used only because we have a large datasetand the memory cannot support this.
#if you have a powerful pc you can remove this bloc
data2 = data2.iloc[:20000]
len(data), len(data2)

(156017, 20000)

In [55]:
#Here we do some processing to remove all english stop words if we have in our data set
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')
#Replace NaN with an empty string
data2['version_Battery_level'] = data2['version_Battery_level'].fillna('')
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(data2['version_Battery_level'])
#Output the shape of tfidf_matrix
tfidf_matrix.shape

(20000, 93)

In [56]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(20000, 20000)

In [57]:
#Construct a reverse map of indices and version_Battery_level
indices = pd.Series(data2.index, index=data2['version_Battery_level']).drop_duplicates()

In [60]:
# Function that takes in version and Battery_level as input and outputs most top 10
def get_recommendations(version,Battery_level, cosine_sim=cosine_sim):
    version_Battery_level = version+'_'+str(Battery_level)
    # Get the index of the version_Battery_level that matches the version_Battery_level
    idx = indices[version_Battery_level]
    
    # Get the pairwsie similarity scores of all version_Battery_level with that version_Battery_level
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the version_Battery_level based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: max(x[1]), reverse=True)

    # Get the scores of the 10 most similar version_Battery_level
    sim_scores = sim_scores[1:6]

    # Get the version_Battery_level indices
    version_Battery_level_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar version_Battery_level
    return data2.iloc[version_Battery_level_indices]

In [61]:
get_recommendations('2.3.1',61.0)

,Userid,UUID,Version,CpuHertz,CPU_0,CPU_1,CPU_2,CPU_3,Total_CPU,TotalMemory_freeSize,...,tot_irq,tot_softirq,ctxt,btime,processes,procs_running,procs_blocked,connectedWifi_SSID,connectedWifi_Level,version_Battery_level
1,97bb95f55a,1462793205551,2.3.1,1.7 GHz,95.2381,5.0,100.0,21.052631,55.32268,1740590.0,...,3118.0,1950376.0,1331527118.0,1462388871.0,8384207.0,4.0,0.0,0x3352A141D7A60F64879A4235D387F36B,-127.0,2.3.1_61.0
2,97bb95f55a,1462793213783,2.3.1,1.7 GHz,72.72727,7.692308,78.94737,4.7619047,41.03221,7874128.0,...,3118.0,1950496.0,1331588413.0,1462388871.0,8384543.0,3.0,0.0,0x3352A141D7A60F64879A4235D387F36B,-127.0,2.3.1_61.0
3,97bb95f55a,1462793218751,2.3.1,1.7 GHz,5.0,4.7619047,4.7619047,4.5454545,4.767316,12478652.0,...,3118.0,1950517.0,1331603005.0,1462388871.0,8384623.0,1.0,0.0,0x3352A141D7A60F64879A4235D387F36B,-127.0,2.3.1_61.0
4,97bb95f55a,1462793223756,2.3.1,1.7 GHz,21.73913,0.0,21.73913,5.0,12.119565,778458.0,...,3118.0,1950554.0,1331623058.0,1462388871.0,8384755.0,3.0,0.0,0x3352A141D7A60F64879A4235D387F36B,-127.0,2.3.1_61.0
5,97bb95f55a,1462793228725,2.3.1,1.7 GHz,9.523809,0.0,0.0,4.7619047,3.5714285,4418888.0,...,3118.0,1950569.0,1331638162.0,1462388871.0,8384857.0,2.0,0.0,0x3352A141D7A60F64879A4235D387F36B,-127.0,2.3.1_61.0
